# Copy selected stereo images, rename them and extract patch

In [3]:
###import libraries
import csv
from pathlib2 import Path

###GLOBAL VARIABLES
str_refFile = '../files/csv/biograd_calm_gestures.csv'


###code

#go through stereo file, and create a list of the images selected for labeling
with open(str_refFile,'r') as stereofile:
    imageReader = csv.reader(stereofile, delimiter=',')
    flag_skip_row = True
    for row in imageReader:
        if not flag_skip_row:
            path_left = Path(row[0])
            path_right = Path(row[1])
            str_left = path_left.name
            str_right = path_right.name
            print(str_left)
        else:
            flag_skip_row = False

stereo_left_0.jpg
stereo_left_1.jpg
stereo_left_10.jpg
stereo_left_100.jpg
stereo_left_1001.jpg
stereo_left_1002.jpg
stereo_left_1003.jpg
stereo_left_1004.jpg
stereo_left_1005.jpg
stereo_left_1006.jpg
stereo_left_1007.jpg
stereo_left_1008.jpg
stereo_left_1009.jpg
stereo_left_101.jpg
stereo_left_1010.jpg
stereo_left_1014.jpg
stereo_left_1015.jpg
stereo_left_1016.jpg
stereo_left_1017.jpg
stereo_left_1018.jpg
stereo_left_1019.jpg
stereo_left_102.jpg
stereo_left_1020.jpg
stereo_left_1021.jpg
stereo_left_1022.jpg
stereo_left_1023.jpg
stereo_left_1024.jpg
stereo_left_1025.jpg
stereo_left_1026.jpg
stereo_left_1027.jpg
stereo_left_1028.jpg
stereo_left_1029.jpg
stereo_left_103.jpg
stereo_left_1030.jpg
stereo_left_1031.jpg
stereo_left_1032.jpg
stereo_left_1033.jpg
stereo_left_104.jpg
stereo_left_1045.jpg
stereo_left_1046.jpg
stereo_left_1047.jpg
stereo_left_1048.jpg
stereo_left_1049.jpg
stereo_left_105.jpg
stereo_left_1050.jpg
stereo_left_1051.jpg
stereo_left_1052.jpg
stereo_left_1053.jpg
stereo